In [4]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import sqlite3

In [5]:
def calc_cosine_sim(model_name, sentence_src, sentence_target):
    model = SentenceTransformer(model_name)
    embedding_src = model.encode(sentence_src, convert_to_tensor=True)
    embedding_target = model.encode(sentence_target, convert_to_tensor=True)
    cosine_score = util.pytorch_cos_sim(embedding_src, embedding_target)
    return cosine_score

In [6]:
conn = sqlite3.connect("../data/clean/correspondence.db")
sql_query = "SELECT * FROM correspondence"
df_corr = pd.read_sql_query(sql_query, conn)
df_corr.head()
# df_exio2fao = df_exio2fao.drop_duplicates(subset=["Item"], keep="first").reset_index(drop=True)

,name_bonsai,name_external,source_external
0,agave fibres nes,agave fibres nes,faostat
1,"alcohol, non-food purposes","alcohol, non-food purposes",faostat
2,"almonds, shelled","almonds, shelled",faostat
3,almonds,almonds,faostat
4,"anise, badian, fennel","anise, badian, fennel",faostat


In [8]:
arr_bonsai= df_corr["name_bonsai"].values
arr_external= df_corr["name_external"].values

cosine_score = calc_cosine_sim("all-mpnet-base-v2", arr_external, arr_bonsai)

In [ ]:
cosine_score.shape

torch.Size([1100, 1100])

In [ ]:
sorted_cs, indices = cosine_score.sort(dim=1, descending=True)

In [ ]:
result_df = pd.DataFrame()

for ix, product in enumerate(arr_external):
    sorted_product_cs = sorted_cs[ix].cpu().numpy()
    exio_ix = indices[ix].cpu().numpy()
    result_df.loc[ix, "description_external"] = product
    result_df.loc[ix, "description_bonsai_nlp"] = df_corr.loc[exio_ix[0], "description_bonsai"]
    result_df.loc[ix, 'cosine_score'] = float("{:.3f}".format(sorted_product_cs[0]))


In [ ]:
result_df.tail()

,description_external,description_bonsai_nlp,cosine_score
1095,"""tide, wave and ocean""",snails o/t sea snails,0.300
1096,wind,electricity,0.496
1097,other sources,other animal products,0.287
1098,electricity,electricity,1.000
1099,heat,electricity,0.362


In [ ]:
df_merge = result_df.merge(
    df_corr, 
    on=["description_external"], 
    how="right",
    ).drop_duplicates()

In [ ]:
len(df_merge), len(df_corr), len(result_df)
df_merge.head()

,description_external,description_bonsai_nlp,cosine_score,description_bonsai,source_external
0,agave fibres nes,agave fibres nes,1.0,agave fibres nes,faostat
1,"alcohol, non-food purposes","alcohol, non-food purposes",1.0,"alcohol, non-food purposes",faostat
2,"almonds, shelled","almonds, shelled",1.0,"almonds, shelled",faostat
3,almonds,almonds,1.0,almonds,faostat
4,"anise, badian, fennel","anise, badian, fennel",1.0,"anise, badian, fennel",faostat


In [ ]:
df_merge["compare"] = df_merge.apply(lambda row: row["description_bonsai_nlp"] == row["description_bonsai"], axis=1)


In [ ]:
len(df_merge[df_merge["compare"]==True])/len(df_merge)

0.5636363636363636

In [ ]:
df_merge.sort_values(by="cosine_score", ascending=True).head(100)

,description_external,description_bonsai_nlp,cosine_score,description_bonsai,source_external,compare
1088,"Remoulade, average values","Vegetables, homogenized preparations",0.213,"Oil, sunflower",Concito,False
1086,"Remoulade, average values","Vegetables, homogenized preparations",0.213,"Eggs, hen, in shell",Concito,False
725,"Dumplings, average values","Beef and Veal, Dried, Salted, Smoked",0.336,Pastry,Concito,False
467,"Aquavit, 40 % vol., average values","Grapefruit Juice, Concentrated",0.357,"Beverages, distilled alcoholic",Concito,False
900,Nougat,"Fruit, fresh nes",0.364,Sugar nes,Concito,False
...,...,...,...,...,...,...
747,"Garlic baguette, frozen","Potatoes, frozen",0.543,Bread,Concito,False
749,"Garlic baguette, frozen","Potatoes, frozen",0.543,Garlic,Concito,False
1217,"Roastbeef, sliced",Turkey meat,0.543,"Meat, cattle",Concito,False
787,"Icetea, peach",Peaches and nectarines,0.543,Food prep nes,Concito,False


In [ ]:
df_merge[df_merge["compare"]==False]

,fao_description,exio_description_nlp,exio_code_nlp,description,Exio prod code,cosine_score,compare
73,Cattle,Cattle - Milk,C_Catmilk,Cattle - Meat (live),C_Catmeat,0.779,False
335,Camels,Fat of camels,C_Fat_cam,Other animal products,C_Othanpr,0.705,False
349,Pigs,Pig meat,C_Meat_pig,Pig meat (live),C_Pigmeat,0.668,False
356,Chickens,Hen eggs,C_Eggsl,Poultry - Meat (live),C_Polmeat,0.682,False
382,Sheep,Sheep - Milk,C_Shemilk,Sheep - Meat (live),C_Shemeat,0.693,False
445,Ducks,Duck meat,C_Meat_duck,Poultry - Meat (live),C_Polmeat,0.562,False
446,Turkeys,Turkey meat,C_Meat_turk,Poultry - Meat (live),C_Polmeat,0.725,False
447,Asses,Meat of asses,C_Meat_ass,Other animal products,C_Othanpr,0.602,False
448,Horses,Offals of horses,C_Offas_hors,Other animal products,C_Othanpr,0.711,False
449,Rabbits,Rabbit meat,C_Meat_rabb,Other animal products,C_Othanpr,0.619,False
